In [289]:
#These imports were used from the cited https://github.com/seanjtaylor/NFLRanking 
import pandas as pd
from sklearn import linear_model
import numpy as np
from patsy import dmatrices, dmatrix

# These imports were my own additions
from numpy import ravel
import operator

### Data from: http://armchairanalysis.com/data.php
The only data publicly available for the 2016 season was for weeks 1-2. Therefore, the following results are based only on the data from weeks 1-2 in season 2016. I could not find any other complete data/files online for NFL seasons for free.

In [290]:
#I got this data from http://armchairanalysis.com/data.php which was cited in https://github.com/seanjtaylor/NFLRanking
#I had to modify this line of code to comply with the new 2016 format name of csv file
game_data = pd.read_csv('nfl_sample_data_2016/GAME.csv', index_col=0)

In [291]:
#This was used from https://github.com/seanjtaylor/NFLRanking
#However, I modified it to work for the new 2016 data format
games = game_data
g16 = games[(games['seas'] == 2016)&(games['wk'] < 18)]

In [292]:
team_data = pd.read_csv('nfl_sample_data_2016/TEAM.csv', index_col=0)
teams = sorted(team_data['tname'].unique())
team_ids = {team: i for i, team in enumerate(teams)}

# I implemented Massey Method below

In [293]:
# I referenced http://public.gettysburg.edu/~cwessell/RankingPage/massey.pdf to implement this function
def massey_method(team_data_input, game_data_input, games_input, teams):
    num_rows = len(games_input)
    num_cols = len(teams)
    matrix = np.zeros([num_rows, num_cols])
    y = np.zeros([num_rows, 1])
    ones_row = np.ones([1, num_cols])
    
    # create mapping between indices of matrix and teams
    team_indices_dict = {}
    indices_team_dict = {}
    team_index = 0
    for team in teams:
        team_indices_dict[team] = team_index
        indices_team_dict[team_index] = team
        team_index += 1
    
    # "In each row, for teams i and j that played each other, there is a 1 in the i space
    # and a −1 in the j space, indicating that these teams played each other, with team i defeating
    # team j, and zeros everywhere else."
    # (http://public.gettysburg.edu/~cwessell/RankingPage/massey.pdf)
    game_index = 0
    for index, row in game_data_input.iterrows():
        if row['ptsh'] > row['ptsv']:
            matrix[team_indices_dict[row['h']]][team_indices_dict[row['v']]] = 1
            matrix[team_indices_dict[row['v']]][team_indices_dict[row['h']]] = -1
        else:
            matrix[team_indices_dict[row['h']]][team_indices_dict[row['v']]] = -1
            matrix[team_indices_dict[row['v']]][team_indices_dict[row['h']]] = 1
        y[game_index] = abs(row['ptsh'] - row['ptsv'])
        game_index += 1
            

    # "Xr = y,
    # where X is our matrix, 
    # r is an n × 1 vector of the ratings we are trying to find, and y is an
    # m × 1 vector of the point differentials of every game."
    # (http://public.gettysburg.edu/~cwessell/RankingPage/massey.pdf)
    
    
    # X_T X r = X_T y
    # XT X becomes a new matrix M 
    # and assign the variable p to XT y, a n × 1 vector of total point differentials
    # (http://public.gettysburg.edu/~cwessell/RankingPage/massey.pdf)
    
    M = np.dot(matrix.T, matrix)
    p = np.dot(matrix.T, y)
    
    # Mr = p.
    # "We are
    # looking for a single solution so we can obtain the rating of each team. To fix this problem,
    # Massey changes the last row of the matrix to a row of all ones and the corresponding entry
    # in the point differential vector to a zero, so that the rank is no longer less than n."
    # (http://public.gettysburg.edu/~cwessell/RankingPage/massey.pdf)
    M[-1,:] = ones_row
    np.put(p, [-1], 0)
    

    ratings = np.linalg.solve(M, p)
    
    # form ratings dictionary
    ratings_dictionary = {}
    
    ratings_index = 0
    for val in ratings:
        ratings_dictionary[indices_team_dict[ratings_index]] = val[0]
        ratings_index += 1

    return ratings_dictionary

The runtime of massey_method(team_data_input, game_data_input) is 

O(ab log(b) + b(a^2))

where 
a = the number of columns in the matrix and
b = the number of rows in the matrix.

In this case, a = number of games in games input and b = number of teams in teams input.

This is because Massey method uses Least Squares, and we know that python's numpy function uses regression, which is

O(m) according to my research and source here: http://komarix.org/ac/papers/thesis/komarek_lr_thesis.pdf

where m is the number of entries in the matrix.


According to this thesis and report, the time complexity varies from O(ba) to O(b(a^2)):
file:///Users/212570554/Downloads/Thesis-Final_Karthik_Iyer.pdf

where the
number of columns (attributes) in each matrix is a and the number of rows (instances) in
each matrix is b.

In [294]:
%%time
massey_method(team_data, game_data, g16, teams)

CPU times: user 7.55 ms, sys: 1.61 ms, total: 9.15 ms
Wall time: 7.26 ms


{'ARI': 35.999999999999886,
 'ATL': 59.999999999999886,
 'BAL': -66.999999999999929,
 'BUF': -69.500000000000256,
 'CAR': 16.999999999999996,
 'CHI': -61.000000000000213,
 'CIN': -80.75000000000027,
 'CLE': 70.250000000000227,
 'DAL': -71.999999999999901,
 'DEN': 48.249999999999915,
 'DET': -41.999999999999893,
 'GB': 19.999999999999943,
 'HOU': -69.999999999999929,
 'IND': -9.9999999999999751,
 'JAC': -24.499999999999844,
 'KC': -49.249999999999822,
 'LA': 31.000000000000018,
 'MIA': 38.000000000000043,
 'MIN': -36.749999999999865,
 'NE': -103.00000000000001,
 'NO': 78.999999999999886,
 'NYG': 89.000000000000227,
 'NYJ': 65.999999999999929,
 'OAK': -96.000000000000156,
 'PHI': 65.999999999999929,
 'PIT': 56.999999999999908,
 'SD': -41.999999999999936,
 'SEA': 89.749999999999972,
 'SF': 75.499999999999943,
 'TB': -102.00000000000007,
 'TEN': -11.999999999999954,
 'WAS': 95.000000000000298}

## I implemented this sorting algorithm to sort the results

In [295]:
def sort_by_val(dict_input):
    sorted_dict_input = sorted(dict_input.items(), key=operator.itemgetter(1))
    return sorted_dict_input

The time complexity of sort_by_val(dict_input) is O(n log n)

This is because python's built-in function sorted is O(n log n) from its documentation because it uses Timsort: https://en.wikipedia.org/wiki/Timsort


I iterate through the n items in the input dict_input in linear time, and then use python's sorted function.

# Wins

In [296]:
#This was used from https://github.com/seanjtaylor/NFLRanking
#However, I modified it to work for the new 2016 data format
def wins(df):
    winner = df.apply(lambda x: x['h'] if x['ptsh'] > x['ptsv'] else x['v'], axis=1)
    w = {team: 31 for team in df['h'].unique()}
    for rank, (team, ws) in enumerate(w.iteritems()):
        w[team] = rank
    return w

The runtime for the function wins(df) is 

O(n)

where n is the number of teams, because it iterates through each of the teams and does constant time O(1) work for each iteration.

In [297]:
%%time
wins16 = wins(g16)
sort_by_val(wins16)

CPU times: user 12.1 ms, sys: 2.12 ms, total: 14.2 ms
Wall time: 3.84 ms


# Pythagorean Wins

#This was used from https://github.com/seanjtaylor/NFLRanking
#However, I modified it to work for the new 2016 data format
pythagorean wins = $\frac{p^{\beta}}{p^{\beta} + q^{\beta}}$

In [298]:
def pythagorean_wins(df, beta=2.37):
    home = df.groupby('h')[['ptsh', 'ptsv']].sum().rename(columns={'ptsh': 'ptsfor', 'ptsv': 'ptsag'}, index={'h': 'team'})
    away = df.groupby('v')[['ptsh', 'ptsv']].sum().rename(columns={'ptsv': 'ptsfor', 'ptsh': 'ptsag'}, index={'v': 'team'})
    newdf = pd.concat([home, away]).groupby(level=0).sum()
    winrate = newdf['ptsfor']**beta / (newdf['ptsfor']**beta + newdf['ptsag']**beta)
    return dict((winrate*-1).rank().astype(int).iteritems())

The runtime for the function pythagorean_wins(df, beta=2.37) is 

O(n)

where n is the number of teams, because it iterates through each of the teams and does constant time O(1) work for each iteration.

In [299]:
%%time
pythagorean_wins16 = pythagorean_wins(g16)
sort_by_val(pythagorean_wins16)

CPU times: user 33.9 ms, sys: 7.61 ms, total: 41.5 ms
Wall time: 12.1 ms


# Eigenvector Methods

Power iteration code comes from [The Glowing Python's blogpost](http://glowingpython.blogspot.com/2011/05/four-ways-to-compute-google-pagerank.html).

PageRank with no damping (damping = 1) should be equivalent to taking the largest eigvenvector.

### My notes:

The following functions under this Eigenvector Methods section were used from 
https://github.com/seanjtaylor/NFLRanking
    
However, I modified it to work for the new 2016 data format

In [300]:
def make_matrix(df, elements=lambda x: (1, 0) if x['ptsh'] > x['ptsv'] else (0, 1)):
    """Turn a set of games into a matrix representation."""
    k = len(teams)
    mat = np.zeros((k, k))
    for ix, row in df.iterrows():
        a, b = elements(row)
        mat[team_ids[row['h']], team_ids[row['v']]] = a
        mat[team_ids[row['v']], team_ids[row['h']]] = b
    return mat

The runtime of the function make_matrix is

O(n)

where n is the number of teams.

This is because the function iterates through the elements of the input and constructs a matrix of its elements.

In [301]:
def pagerank(g, damping=1):
    values = g.pagerank(damping=damping)
    g_pr = {vert['name']: val for vert, val in zip(g.vs, values)}
    return g_pr #{key: i for i, (key, v) in enumerate(sorted(g_pr.iteritems(), key=lambda x: x[1]))}

The runtime of pagerank is

O(n^2) in the worst case, where n is the length of the vector, as each iteration of pagerank multiplies the matrix with the vector

according to this research: http://www.ee.iitm.ac.in/~ee11b075/files/dsa-termpaper.pdf

In [302]:
def power_iteration(m, iters=10000, x0=None):
    if x0 == None:
        x0 = np.ones(m.shape[0])
        x0 /= np.linalg.norm(x0, 1)

    for i in range(iters):
        x0 = np.dot(m,x0)
        x0 /= np.linalg.norm(x0,1)

    scores = {team: x0[i] for team, i in team_ids.iteritems()}
    return {team: rank for rank, (team, score) in enumerate(sorted(scores.iteritems(), key=lambda x: x[1], reverse=True))}


The runtime of power_iteration() is

O(r*c + n) where r is the size of each row, and c is the size of each column, in the input matrix m. And n is the number of teams.

This is because power_iteration() performs the dot product of the matrix and the vector. It also performs linear computations as it iterates through the n teams.

In [303]:
def eigen_rank(df):
    m = make_matrix(df)
    return power_iteration(m)

The runtime of eigen_rank(df) is

O(n + r*c)

Because it calls make_matrix(df) which runs in O(n) where n is the number of teams
and it also calls power_iteration(m) which runs in O(r*c + n) where r is the size of each row, and c is the size of each column, in the input matrix m. And n is the number of teams.

In [304]:
def eigen_rank_scores(df):
    """Uses points scored instead of dummy variables for wins"""
    m = make_matrix(df, elements=lambda x: (x['ptsh'], x['ptsv']))
    return power_iteration(m)

The runtime of eigen_rank_scores(df) is 

O(n + r*c)

Because it calls make_matrix(df) which runs in O(n) where n is the number of teams
and it also calls power_iteration(m) which runs in O(r*c + n) where r is the size of each row, and c is the size of each column, in the input matrix m. And n is the number of teams.

In [305]:
def score_transform(s1, s2):
    x = (s1 + 1.0) / (s1 + s2 + 2.0)
    return 0.5 + 0.5 * np.sign(x - 0.5) * np.sqrt(np.abs(2*x - 1))

The runtime of score_transform(s1, s2) is

O(n) where n is the length of s1/s2 because it performs computations on these vectors in constant time.
In this case, n is the number of teams.

In [306]:
def eigen_rank_scores_nonlinear(df):
    """Uses a transformed version of score with gives less credit for blowouts
    See Keener (1993).
    """ 
    m = make_matrix(df, elements=lambda x: (score_transform(x['ptsh'], x['ptsv']), score_transform(x['ptsv'], x['ptsh'])))
    return power_iteration(m)

The runtime of eigen_rank_scores_nonlinear(df) is

O(n + r*c)

Because it calls make_matrix(df) which runs in O(n) where n is the number of teams
and it also calls power_iteration(m) which runs in O(r*c + n) where r is the size of each row, and c is the size of each column, in the input matrix m. And n is the number of teams.

In [307]:
%%time
eigen_rank_scores_nonlinear(g16)

CPU times: user 311 ms, sys: 30.1 ms, total: 341 ms
Wall time: 132 ms


{'ARI': 6,
 'ATL': 20,
 'BAL': 25,
 'BUF': 31,
 'CAR': 2,
 'CHI': 18,
 'CIN': 28,
 'CLE': 27,
 'DAL': 26,
 'DEN': 1,
 'DET': 5,
 'GB': 12,
 'HOU': 13,
 'IND': 10,
 'JAC': 16,
 'KC': 14,
 'LA': 11,
 'MIA': 7,
 'MIN': 4,
 'NE': 0,
 'NO': 24,
 'NYG': 21,
 'NYJ': 29,
 'OAK': 22,
 'PHI': 17,
 'PIT': 23,
 'SD': 15,
 'SEA': 9,
 'SF': 3,
 'TB': 19,
 'TEN': 8,
 'WAS': 30}

# Bradley-Terry (and Rasch) models, from Keener

In [308]:
#This was used from https://github.com/seanjtaylor/NFLRanking
#However, I modified it to work for the new 2016 data format
teams = sorted(games['h'].unique())

def make_model_matrix(df):
    dummies = {team: (df['h'] == team).astype(np.int) - (df['v'] == team).astype(np.int)
               for team in teams}
    df2 = pd.DataFrame(dummies)
    df2['win'] = (df['ptsh'] > df['ptsv']).astype(np.int)

    y, X = dmatrices('win ~ 0 + %s' % ' + '.join(teams), df2)
    y = ravel(y)
    return y, X

The runtime of the function make_model_matrix(df) is

O(n)

where n is the number of teams.

This is because the function constructs dummies by looping through the n teams: O(n)
Then, it calls other linear and constant time functions: O(n)

In [309]:
#This was used from https://github.com/seanjtaylor/NFLRanking
def logit_rank(df, c=1.0):
    y, X = make_model_matrix(df)
    clf = linear_model.LogisticRegression(C=c, penalty='l2', tol=1e-6, fit_intercept=False)
    clf.fit(X, y)
    return {team: rank for rank, (team, score) in enumerate(sorted(zip(teams, clf.coef_[0]), key=lambda x: x[1], reverse=True))}


The runtime of the function logit_rank(df, c=1.0) is

O(ab log(b) + b(a^2))

where 
a = the number of columns in the matrix and
b = the number of rows in the matrix.



This is because the first call to make_model_matrix(df) is O(n), explained above, where n is the number of teams. The matrix is created in this call.


Then, linear_model.LogisticRegression() is O(m) according to my research and source here: http://komarix.org/ac/papers/thesis/komarek_lr_thesis.pdf

where m is the number of entries in the matrix.


According to this thesis and report, the time complexity varies from O(ba) to O(b(a^2)):
file:///Users/212570554/Downloads/Thesis-Final_Karthik_Iyer.pdf

where the
number of columns (attributes) in each matrix is a and the number of rows (instances) in
each matrix is b.


The clf.fit(X, y) is O(ab log(b))

where a = number of features
and b = number of samples
according to the documentation: http://scikit-learn.org/stable/modules/tree.html#complexity
X has size [n_samples, n_features] while y has size [n_samples].

In [310]:
%%time
logit_rank_result_2016 = logit_rank(g16)
logit_rank_result_2016

CPU times: user 53.7 ms, sys: 1.95 ms, total: 55.7 ms
Wall time: 56.4 ms


The time complexity of sort_by_rank(team_rank_dict) is O(n log n)

This is because python's built-in function sorted is O(n log n) from its documentation because it uses Timsort: https://en.wikipedia.org/wiki/Timsort


In [311]:
%%time
sorted_logit_rank_result_2016 = sort_by_val(logit_rank_result_2016)
sorted_logit_rank_result_2016

CPU times: user 22 µs, sys: 2 µs, total: 24 µs
Wall time: 29.1 µs


## I implemented this algorithm to sort the teams by their points

In [312]:
#I got this data from http://armchairanalysis.com/data.php which was cited in https://github.com/seanjtaylor/NFLRanking
team_data = pd.read_csv('nfl_sample_data_2016/TEAM.csv', index_col=0)

def sort_by_points_total(df):
    team_names = df['tname']
    team_points = df['pts']
    
    teams_by_points = {}
    
    # initialize team_by_points dictionary
    for name in team_names:
        teams_by_points[name] = 0
    
    for name, point in zip(team_names, team_points):
        teams_by_points[name] += point
    
    sorted_teams_by_points = sort_by_val(teams_by_points)
    
    return sorted_teams_by_points

The runtime of sort_by_points_total(df) is O(n log n)

This is because python's built-in function sorted is O(n log n) from its documentation because it uses Timsort: https://en.wikipedia.org/wiki/Timsort

I iterate through the n teams in linear time, and then use python's sorted function.

In [313]:
%%time
sort_by_points_total(team_data)

CPU times: user 288 µs, sys: 39 µs, total: 327 µs
Wall time: 307 µs


[('LA', 9),
 ('SEA', 15),
 ('CHI', 28),
 ('CLE', 30),
 ('TEN', 32),
 ('MIA', 34),
 ('NYG', 36),
 ('JAC', 37),
 ('BAL', 38),
 ('TB', 38),
 ('BUF', 38),
 ('CIN', 39),
 ('WAS', 39),
 ('GB', 41),
 ('MIN', 42),
 ('HOU', 42),
 ('KC', 45),
 ('DAL', 46),
 ('NO', 47),
 ('DET', 54),
 ('DEN', 55),
 ('IND', 55),
 ('SF', 55),
 ('PHI', 58),
 ('ATL', 59),
 ('NYJ', 59),
 ('ARI', 61),
 ('PIT', 62),
 ('OAK', 63),
 ('SD', 65),
 ('CAR', 66),
 ('NE', 81)]

## I implemented this algorithm to sort the teams by their points

In [314]:
#I got this data from http://armchairanalysis.com/data.php which was cited in https://github.com/seanjtaylor/NFLRanking
team_data = pd.read_csv('nfl_sample_data_2016/TEAM.csv', index_col=0)

def sort_by_points_latest_week(df):
    team_names = df['tname']
    team_points = df['pts']
    
    teams_by_points = {}
    
    # initialize team_by_points dictionary
    for name in team_names:
        teams_by_points[name] = 0
    
    for name, point in zip(team_names, team_points):
        teams_by_points[name] = point # latest week replaces all other previous week values
    
    sorted_teams_by_points = sort_by_val(teams_by_points)
    
    return sorted_teams_by_points

The runtime of sort_by_points(df) is O(n log n)

This is because python's built-in function sorted is O(n log n) from its documentation because it uses Timsort: https://en.wikipedia.org/wiki/Timsort

I iterate through the n teams in linear time, and then use python's sorted function.

In [315]:
%%time
sort_by_points_latest_week(team_data)

CPU times: user 576 µs, sys: 39 µs, total: 615 µs
Wall time: 1.4 ms


[('HOU', 0),
 ('SEA', 3),
 ('TB', 7),
 ('LA', 9),
 ('KC', 12),
 ('NO', 13),
 ('CHI', 14),
 ('GB', 14),
 ('JAC', 14),
 ('DET', 15),
 ('CIN', 16),
 ('NYG', 16),
 ('TEN', 16),
 ('MIN', 17),
 ('CLE', 20),
 ('IND', 20),
 ('WAS', 23),
 ('MIA', 24),
 ('PIT', 24),
 ('BAL', 25),
 ('DAL', 27),
 ('NE', 27),
 ('SF', 27),
 ('OAK', 28),
 ('PHI', 29),
 ('BUF', 31),
 ('DEN', 34),
 ('ATL', 35),
 ('NYJ', 37),
 ('SD', 38),
 ('ARI', 40),
 ('CAR', 46)]